Read in JSON files containing the dialogs

In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install jsonlines


Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
from pandas.io.json import json_normalize
import os
import logging
import logging.handlers
import pandas as pd
import numpy as np
import glob
from datetime import datetime,timedelta
from collections import Counter

In [4]:
def dt_parse(t):
    ret = datetime.strptime(t[0:16],'%Y-%m-%dT%H:%M')
    if t[18]=='+':
        ret-=timedelta(hours=int(t[19:22]),minutes=int(t[23:]))
    elif t[18]=='-':
        ret+=timedelta(hours=int(t[19:22]),minutes=int(t[23:]))
    return ret 

# Nasoto DataFrame

In [5]:
rootDirNasoto = 'webchat-nasoto-pilot/' 

jsdata_nasoto = []

df_nasoto = pd.DataFrame(pd.np.empty((0, 6)))
df_nasoto.columns = ['Username','Duration', 'Chatbot', 'NumberOfTurns', 'Utterance','Length of Utterances']

for subdir, dirs, files in os.walk(rootDirNasoto): 
    for dire in dirs:
        arrTime = []
        arrUtterance = []
        numberOfTurns = 0
        count = 0
        for file in glob.glob(rootDirNasoto+dire+'/'+'*.json'):
            with open(file) as f:
                data = json.load(f)
                numberOfTurns += 1
                arrTime.append( dt_parse(data['timestamp']))
                arrUtterance.append(data['text'])
                for word in (data['text']).split():
                    count += 1
                
        
        
        #add post interaction survey (Nasoto)
        
        #Duration
        maxTime = max(dt for dt in arrTime )
        minTime = min(dt for dt in arrTime)
        duration =  maxTime - minTime
        
        #get the information from makoto inserted
        jsdata_nasoto.append({'Username': data['from']['name'],
                       'Duration': duration,'Chatbot': data['recipient']['name'], 
                       'NumberOfTurns' : numberOfTurns, 'Utterance': arrUtterance, 'Length of Utterances': count})



df_nasoto =  pd.DataFrame.from_records(jsdata_nasoto)
df_nasoto[:]['Duration'] / np.timedelta64(1, 'm')
df_nasoto  

/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """


,Username,Duration,Chatbot,NumberOfTurns,Utterance,Length of Utterances
0,sorcha.nicconmara,00:08:00,Nasoto-Bot,19,"[I now study from home, I think it was the rig...",68
1,charles.kelly1,00:07:00,Nasoto-Bot,42,"[Playing basketball, No thank you, No thank yo...",101
2,chloe.foxe,00:04:00,Nasoto-Bot,23,"[I am taking 5 modules this semester, Nah, ext...",84
3,elizabeth.burke,00:03:00,Nasoto-Bot,17,"[12 modules, no, no, 12, It's fun, my favourit...",48


# Makoto DataFrame

In [6]:
rootDirMakoto = 'webchat-makoto-pilot/'
jsdata_makoto = []

df_makoto = pd.DataFrame(pd.np.empty((0, 5)))
df_makoto.columns = ['Username','Duration', 'Chatbot', 'NumberOfTurns', 'Utterance']

for subdir, dirs, files in os.walk(rootDirMakoto): 
    for dire in dirs:
        arrTime = []
        arrUtterance = []
        numberOfTurns = 0
        count = 0
        for file in glob.glob(rootDirMakoto+dire+'/'+'*.json'):
            with open(file) as f:
                data = json.load(f)
                numberOfTurns += 1
                arrTime.append( dt_parse(data['timestamp']))
                arrUtterance.append(data['text'])
                for word in (data['text']).split():
                    count += 1
        
        #information for the google form
        #add post interaction survey (Makoto)
        #Duration
        maxTime = max(dt for dt in arrTime )
        minTime = min(dt for dt in arrTime)
        duration =  maxTime - minTime
        
        #get the information from makoto inserted
        jsdata_makoto.append({'Username': data['from']['name'],
                       'Duration': duration,'Chatbot': data['recipient']['name'], 
                       'NumberOfTurns' : numberOfTurns, 'Utterance': arrUtterance, 'Length of Utterances': count})
df_makoto =  pd.DataFrame.from_records(jsdata_makoto) 
df_makoto[:]['Duration'] / np.timedelta64(1, 'm')
df_makoto

/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.


,Username,Duration,Chatbot,NumberOfTurns,Utterance,Length of Utterances
0,chloe.foxe,00:05:00,Makoto-Bot,10,"[Uhm. I like solving problems, yeah we have a ...",46
1,elizabeth.burke,00:09:00,Makoto-Bot,29,"[It's an inconvenience but necessary, health i...",88
2,charles.kelly1,00:02:00,Makoto-Bot,11,"[Yes ther is, Yes, I like animals, 4 or 5, Gam...",26
3,sorcha.nicconmara,00:05:00,Makoto-Bot,15,"[extracurricular, The topic is very interestin...",39


DataFrame for Pre-Interacrion Survey

In [7]:
PreInteractionDir = 'surveys-pilot/Pre-InteractionSurvey-Pilot.csv'
df_Survey1 = pd.read_csv(PreInteractionDir,  error_bad_lines=False)


Clean Data

In [9]:
df_Survey1 = df_Survey1.drop([ "Total score", "Age [Score]", "Age [Feedback]", 
                              "Gender [Score]" , "Gender [Feedback]", "Do you know what a chatbot is? [Score]", 
                              "Do you know what a chatbot is? [Feedback]", 
                              "If Yes, describe your understanding of what a chatbot is  [Score]",
                            "If Yes, describe your understanding of what a chatbot is  [Feedback]", "How often do you use Siri, Alexa, Google Assistant or any other virtual assistant? [Score]",
                            "How often do you use Siri, Alexa, Google Assistant or any other virtual assistant? [Feedback]",
                             "I am reserved [Score]", "I am reserved [Feedback]", "I tend to find fault with others [Score]",
                            "I tend to find fault with others [Feedback]", "I am helpful and unselfish with others [Score]",
                            "I am helpful and unselfish with others [Feedback]", "I am full of energy [Score]",
                            "I am full of energy [Feedback]", "I start quarrels with others [Score]",
                            "I start quarrels with others [Feedback]", "I generate a lot of enthusiasm  [Score]",
                            "I generate a lot of enthusiasm  [Feedback]","I have a forgiving nature [Score]",
                            "I have a forgiving nature [Feedback]", "I tend to be quiet  [Score]",
                            "I tend to be quiet  [Feedback]", "I am generally trusting [Score]",
                            "I am generally trusting [Feedback]", "I have an assertive personality [Score]",
                            "I have an assertive personality [Feedback]", "I can be cold and aloof  [Score]",
                            "I can be cold and aloof  [Feedback]", "I am sometimes shy, inhibited [Score]",
                            "I am sometimes shy, inhibited [Feedback]", "I am considerate and kind to almost everyone [Score]",
                            "I am considerate and kind to almost everyone [Feedback]", "I am outgoing, sociable  [Score]",
                            "I am outgoing, sociable  [Feedback]", "I am sometimes rude to others [Score]",
                            "I am sometimes rude to others [Feedback]", "I like to cooperate with others [Score]",
                            "I like to cooperate with others [Feedback]"], axis=1)

In [10]:
df_Survey1

,Timestamp,Username,Age,Gender,Do you know what a chatbot is?,"If Yes, describe your understanding of what a chatbot is","How often do you use Siri, Alexa, Google Assistant or any other virtual assistant?",I am reserved,I tend to find fault with others,I am helpful and unselfish with others,...,I have a forgiving nature,I tend to be quiet,I am generally trusting,I have an assertive personality,I can be cold and aloof,"I am sometimes shy, inhibited",I am considerate and kind to almost everyone,"I am outgoing, sociable",I am sometimes rude to others,I like to cooperate with others
0,2020/04/01 10:45:16 am CET,sorcha.nicconmara@ucdconnect.ie,18-24,Female,Yes,A program/piece of software that imitates huma...,Never,2,3,4,...,4,2,4,4,1,4,4,4,3,4
1,2020/04/01 1:45:07 pm CET,charles.kelly1@ucdconnect.ie,18-24,Male,Yes,An ai that attempts to replicate human convers...,Somewhat,4,5,4,...,3,4,3,2,4,4,4,4,4,4
2,2020/04/01 3:11:18 pm CET,chloe.foxe@ucdconnect.ie,18-24,Female,Yes,A chatbot is a computer program with human cap...,Somewhat,4,2,4,...,2,4,5,4,5,3,4,3,3,5
3,2020/04/01 6:44:12 pm CET,elizabeth.burke@ucdconnect.ie,18-24,Female,Yes,Artificial Intelligence capable of holding a c...,Somewhat,2,3,5,...,2,2,4,4,1,1,5,5,1,5


DataFrame for Post-Interaction Survey (Makoto)

In [11]:
PostMakotoDir = 'surveys-pilot/Post-InteractionSurveyMakoto-Pilot.csv'
df_PostMakoto = pd.read_csv(PostMakotoDir, error_bad_lines=False)
df_PostMakoto

,Timestamp,Username,Describe Makoto,How would you rate your experience with Makoto based on its knowledge?,Expand on your answer (optional),How would you rate your experience with Makoto based on the quality of the conversation?,Expand on your answer (optional).1,How would you rate your experience with Makoto based on the its attitude/personality?,Expand on your answer (optional).2,Did you enjoy your interaction with Makoto?
0,2020/04/01 11:07:27 am CET,sorcha.nicconmara@ucdconnect.ie,enthusiastic and lively,4,NaN,4,Interpreted 'okay' as a negative ('no') at one...,4,NaN,yes
1,2020/04/01 2:00:34 pm CET,charles.kelly1@ucdconnect.ie,Cheery,3,Didnt seem to present any knowledge. Didnt kno...,4,The brief conversation was quite pleasant but ...,4,Pleasant personality,For the brief time yes
2,2020/04/01 3:21:27 pm CET,chloe.foxe@ucdconnect.ie,Makoto was inquisitive. It reflected on my ans...,3,it knew about the modules I took. Didn't under...,3,Would've have been exceptional only for the fa...,5,Nice bot. Postive and engaged with my responses.,Would've preferred if it opened up and told me...
3,2020/04/01 7:02:07 pm CET,elizabeth.burke@ucdconnect.ie,"It's an interesting chatbot, the UI was overal...",3,"Interesting topic and conversation arc, but go...",3,"Topics were good, transitions between topics w...",3,It didn't really have a clear personality to b...,Yeah


DataFrame for Post-Interaction Survey (Nasoto)

In [12]:
SurveyNasotoDir = 'surveys-pilot/Post-InteractionSurveyNasoto-Pilot.csv'
df_surveyNasoto = pd.read_csv(SurveyNasotoDir)
df_surveyNasoto

,Timestamp,Username,Describe Nasoto,How would you rate your interaction/experience with Nasoto based on its knowledge?,Expand on your answer (optional),How would you rate your experience/interaction with the Nasoto based on the quality of the conversation?,Expand on your answer (optional).1,How would you rate your experience/interaction with Nasoto based on the it's attitude/personality?,Expand on your answer (optional).2,Did you enjoy your interaction with Nasoto?
0,2020/04/01 10:58:46 am CET,sorcha.nicconmara@ucdconnect.ie,Positive - would try to see the other side of ...,5,NaN,4,One case where Nasoto didn't equate 'sure' wit...,4,NaN,yes
1,2020/04/01 1:56:24 pm CET,charles.kelly1@ucdconnect.ie,Robotic,4,It seemed to be well informed on UCD,1,Wouldnt recognize my answers. Got stuck in a l...,3,Very neutral. No real personality. Neither bad...,Moderately. It was interesting
2,2020/04/01 3:30:06 pm CET,chloe.foxe@ucdconnect.ie,I liked how the bot spoke about it's own perso...,4,NaN,4,He was more knowledgeable than Makoto,4,"Sometimes he didnt understand me. But overall,...","yes, it was nice reflecting on college life a..."
3,2020/04/01 7:11:02 pm CET,elizabeth.burke@ucdconnect.ie,Standoffish? If you can even accuse a chatbot ...,2,NaN,2,NaN,1,"It was very formal, and was noticeably less 'h...",Not really


 DataFrame for Post-Interaction Survey/Final Survey (Both) 

In [13]:
Survey4Dir = 'surveys-pilot/FinalSurvey-Pilot.csv'
df_survey4 = pd.read_csv(Survey4Dir)
df_survey4

,Timestamp,Username,Did you notice any differences in your interactions with one chatbot over the other? If so what were they.,Which chatbot did you prefer interacting with?,Explain why you preferred interacting with this chatbot?
0,2020/04/01 11:08:44 am CET,sorcha.nicconmara@ucdconnect.ie,Makoto was more upbeat and enthusiastic than ...,Makoto,felt that Makoto was very encouraging and 'hap...
1,2020/04/01 2:01:23 pm CET,charles.kelly1@ucdconnect.ie,Nasato had no personality. Makoto stopped work...,Makoto,Seemed more human
2,2020/04/01 3:33:01 pm CET,chloe.foxe@ucdconnect.ie,Nasoto opened up about his likes/dislikes more...,Nasoto,I preferred interacting with nasoto because I ...
3,2020/04/01 7:08:03 pm CET,elizabeth.burke@ucdconnect.ie,"The tone, the second one felt far more automat...",Makoto,It was a nicer Ux because of the conversationa...


Automation of personality assignment

In [14]:
extraversion = ((6 - df_Survey1['I am reserved']) 
        + df_Survey1['I am full of energy'] 
        + df_Survey1['I generate a lot of enthusiasm '] 
        + (6 - df_Survey1['I tend to be quiet ']) 
        + df_Survey1['I have an assertive personality'] 
        + (6 - df_Survey1['I am sometimes shy, inhibited']) 
        + df_Survey1['I am outgoing, sociable '])


df_Survey1["extraversion score"] = extraversion
               
               
agreeableness = ((6 - df_Survey1['I tend to find fault with others']) 
        + df_Survey1['I am helpful and unselfish with others']
        + (6 - df_Survey1['I start quarrels with others'] )
        + df_Survey1['I have a forgiving nature'] 
        + df_Survey1['I am generally trusting'] 
        + (6 - df_Survey1['I can be cold and aloof '])
        + df_Survey1['I am considerate and kind to almost everyone']
        + ( 6 - df_Survey1['I am sometimes rude to others']) 
        + df_Survey1['I like to cooperate with others'])
df_Survey1["agreeableness score"] = agreeableness




In [15]:
df_Survey1

,Timestamp,Username,Age,Gender,Do you know what a chatbot is?,"If Yes, describe your understanding of what a chatbot is","How often do you use Siri, Alexa, Google Assistant or any other virtual assistant?",I am reserved,I tend to find fault with others,I am helpful and unselfish with others,...,I am generally trusting,I have an assertive personality,I can be cold and aloof,"I am sometimes shy, inhibited",I am considerate and kind to almost everyone,"I am outgoing, sociable",I am sometimes rude to others,I like to cooperate with others,extraversion score,agreeableness score
0,2020/04/01 10:45:16 am CET,sorcha.nicconmara@ucdconnect.ie,18-24,Female,Yes,A program/piece of software that imitates huma...,Never,2,3,4,...,4,4,1,4,4,4,3,4,25,36
1,2020/04/01 1:45:07 pm CET,charles.kelly1@ucdconnect.ie,18-24,Male,Yes,An ai that attempts to replicate human convers...,Somewhat,4,5,4,...,3,2,4,4,4,4,4,4,16,27
2,2020/04/01 3:11:18 pm CET,chloe.foxe@ucdconnect.ie,18-24,Female,Yes,A chatbot is a computer program with human cap...,Somewhat,4,2,4,...,5,4,5,3,4,3,3,5,23,31
3,2020/04/01 6:44:12 pm CET,elizabeth.burke@ucdconnect.ie,18-24,Female,Yes,Artificial Intelligence capable of holding a c...,Somewhat,2,3,5,...,4,4,1,1,5,5,1,5,30,38


In [37]:
### Merge all the surveys based on username (email)
from functools import reduce

In [39]:
df_allSurveys =  [df_Survey1, df_PostMakoto, df_surveyNasoto, df_survey4]


df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Username'],
                                            how='outer'), df_allSurveys)


,Timestamp_x,Username,Age,Gender,Do you know what a chatbot is?,"If Yes, describe your understanding of what a chatbot is","How often do you use Siri, Alexa, Google Assistant or any other virtual assistant?",I am reserved,I tend to find fault with others,I am helpful and unselfish with others,...,Expand on your answer (optional)_y,How would you rate your experience/interaction with the Nasoto based on the quality of the conversation?,Expand on your answer (optional).1_y,How would you rate your experience/interaction with Nasoto based on the it's attitude/personality?,Expand on your answer (optional).2_y,Did you enjoy your interaction with Nasoto?,Timestamp_y,Did you notice any differences in your interactions with one chatbot over the other? If so what were they.,Which chatbot did you prefer interacting with?,Explain why you preferred interacting with this chatbot?
0,2020/04/01 10:45:16 am CET,sorcha.nicconmara@ucdconnect.ie,18-24,Female,Yes,A program/piece of software that imitates huma...,Never,2,3,4,...,NaN,4,One case where Nasoto didn't equate 'sure' wit...,4,NaN,yes,2020/04/01 11:08:44 am CET,Makoto was more upbeat and enthusiastic than ...,Makoto,felt that Makoto was very encouraging and 'hap...
1,2020/04/01 1:45:07 pm CET,charles.kelly1@ucdconnect.ie,18-24,Male,Yes,An ai that attempts to replicate human convers...,Somewhat,4,5,4,...,It seemed to be well informed on UCD,1,Wouldnt recognize my answers. Got stuck in a l...,3,Very neutral. No real personality. Neither bad...,Moderately. It was interesting,2020/04/01 2:01:23 pm CET,Nasato had no personality. Makoto stopped work...,Makoto,Seemed more human
2,2020/04/01 3:11:18 pm CET,chloe.foxe@ucdconnect.ie,18-24,Female,Yes,A chatbot is a computer program with human cap...,Somewhat,4,2,4,...,NaN,4,He was more knowledgeable than Makoto,4,"Sometimes he didnt understand me. But overall,...","yes, it was nice reflecting on college life a...",2020/04/01 3:33:01 pm CET,Nasoto opened up about his likes/dislikes more...,Nasoto,I preferred interacting with nasoto because I ...
3,2020/04/01 6:44:12 pm CET,elizabeth.burke@ucdconnect.ie,18-24,Female,Yes,Artificial Intelligence capable of holding a c...,Somewhat,2,3,5,...,NaN,2,NaN,1,"It was very formal, and was noticeably less 'h...",Not really,2020/04/01 7:08:03 pm CET,"The tone, the second one felt far more automat...",Makoto,It was a nicer Ux because of the conversationa...


# Summary Stats

In [109]:
pip install plotly==4.6.0

Note: you may need to restart the kernel to use updated packages.


In [110]:
import plotly.express as px

In [115]:
#merge df_nasoto with df_makoto
df_cd = df_nasoto.append(df_makoto)
# convert duratin to minutes from ns
df_cd = df_cd.astype({'Duration': 'timedelta64[m]'})

## Box Plot  Interaction Duration

In [116]:
fig = px.box(df_cd , "Chatbot", "Duration", "Chatbot")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

## BoxPlot Number of Turns

In [117]:
fig = px.box(df_cd , "Chatbot", "NumberOfTurns", "Chatbot")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

## BoxPlot Length of Utterances

In [118]:
fig = px.box(df_cd , "Chatbot", "Length of Utterances", "Chatbot")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

# RQ1: Does the personality demonstrated by a chatbot affect the user’s experience?


- Rating of experience based on the attitude/personality of the chatbot
- Anlayse perferred chatbot
- Explain why you preferred interacting with this chatbot? 
- Compare Duration (time & # of turns) for each chatbot
- Lenghth of utterances
- Look at whether their personality matches that of the preffered chatbot 




### Check if there's a correlation between the preffered chatbot and personality scores

#### Label data scores for each person and labeled with perferred chatbot

#### TO DO: try predict what chatbot they would prefer based on their scores

In [42]:
df_personality = pd.DataFrame(pd.np.empty((0, 3)))
df_personality.columns = ['Extraversion Score', 'Agreeableness Score', 'Preffered Chatbot']

### using df_merged
df_personality['Extraversion Score'] = df_merged['extraversion score']
df_personality['Agreeableness Score']= df_merged['agreeableness score']
df_personality['Preffered Chatbot'] =  df_merged['Which chatbot did you prefer interacting with?']

df_personality

/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


,Extraversion Score,Agreeableness Score,Preffered Chatbot
0,25,36,Makoto
1,16,27,Makoto
2,23,31,Nasoto
3,30,38,Makoto


# RQ2: Can Personality be simulated by a chatbot?

- Analyse any descriptions of the chatbots for key words associated with personality
- look at response for: How would you rate your experience with Makoto based on the its attitude/personality? + expansion



# RQ3: Can a user’s personality be inferred through their interaction with a chatbot?

- NLP on the language used in all utterances (for both Chatbots)
- Compare personality assigned through test with these results